In [16]:
from heapq import heappop, heappush
import numpy as np

In [18]:
def PC_CBS(gridMap, Starts, Goals, vertexCons, edgeCons): # vertex/edge- Cons - нужны только при вызове CBS в качестве нижнего уровня MACBS
    root = HighNode()
    OPEN = OpenHigh()
    CLOSED = ClosedHigh()
    agents = list(range(len(Starts)))
    k = 0
    
    for a in agents:
        planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], [], [])
        result = planner.FindPath()
        if result[0]:
            path = MakePath(result[1])
            root.sol[a], _ = path
        else:
            return (False, None, OPEN, CLOSED)
    
    root.g = sum([len(path) for path in root.sol.values()])
    OPEN.AddNode(root)
    k += 1
    
    while not OPEN.isEmpty():
        s = OPEN.GetBestNode(CLOSED)
        CLOSED.AddNode(s)  
        newVertexCons = []
        newEdgeCons = []
        
        IsCardinal = False
        index = 0
        for a in agents:
            for b in agents[a + 1 :]:
                for step in range(min(len(s.sol[a]), len(s.sol[b]))):
                    if s.sol[a][step].i == s.sol[b][step].i and s.sol[a][step].j == s.sol[b][step].j:
                        newVertexCons.append((a, b, (s.sol[a][step].i, s.sol[a][step].j), step))
                        
                                                
                    if step + 1 < min(len(s.sol[a]), len(s.sol[b])) and \
                       s.sol[a][step].i == s.sol[b][step + 1].i and s.sol[a][step].j == s.sol[b][step + 1].j and \
                       s.sol[a][step + 1].i == s.sol[b][step].i and s.sol[a][step + 1].i == s.sol[b][step].i:
                        newEdgeCons.append((
                            a,
                            b,
                            (s.sol[a][step].i, s.sol[a][step].j),
                            (s.sol[a][step + 1].i, s.sol[a][step + 1].j),
                            step,
                        ))
                        
        if len(newVertexCons) == 0 and len(newEdgeCons) == 0:
            return (True, s, OPEN, CLOSED)
        
        # Сейчас сначала разрешаются вершинные конфликты, потом реберные
        if len(newVertexCons) > 0:
            while not IsCardinal:
                a, b, (i, j), t = newVertexCons[index]
                
                FlagChild1 = False
                FlagChild2 = False
                
                Child1_not_expanded = False
                Child2_not_expanded = False
            
                # Разбиваем CT на ноды A и B, разрешая вершинный конфликт 
            
                tmp = s.vertexCons.copy()
                if a in tmp:
                    tmp[a].append(((i, j), t))   
                else:
                    tmp[a] = [((i, j), t)]
            
                A = HighNode(
                    vertexCons=tmp,
                    edgeCons=s.edgeCons,
                    sol=s.sol,
                    parent=s,
                    k=k,
                )
            
                ec = []
                if a in A.edgeCons:
                    ec = A.edgeCons[a]
            
                planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], A.vertexCons[a], ec)
                result = planner.FindPath()
                if result[0]:
                    path = MakePath(result[1])
                    A.sol[a], _ = path
                    A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F

                    if  A.g > s.g:
                        FlagChild1 = True #OPEN.AddNode(A)
                    if not CLOSED.WasExpanded(A):
                        Child1_not_expanded = True
                    k += 1
                
                tmp = s.vertexCons.copy()
                if b in tmp:
                    tmp[b].append(((i, j), t))   
                else:
                    tmp[b] = [((i, j), t)]
                
                B = HighNode(
                    vertexCons=tmp,
                    edgeCons=s.edgeCons,
                    sol=s.sol,
                    parent=s,
                    k=k,
                )
            
                ec = []
                if b in B.edgeCons:
                    ec = B.edgeCons[b]
            
                planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], B.vertexCons[b], ec)
                result = planner.FindPath()
                if result[0]:
                    path = MakePath(result[1])
                    B.sol[b], _ = path
                    B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                    if  B.g > s.g:
                        FlagChild2 = True #OPEN.AddNode(B)
                    if not CLOSED.WasExpanded(B):
                        Child2_not_expanded = True
                    k += 1
                if FlagChild1 and FlagChild2:
                    IsCardinal = True
                    if  Child1_not_expanded and  Child2_not_expanded:
                        OPEN.AddNode(A)
                        OPEN.AddNode(B)
                elif index == len(newVertexCons)-1:
                    if  Child1_not_expanded and  Child2_not_expanded:
                        OPEN.AddNode(A)
                        OPEN.AddNode(B)
                else: 
                    index+=1
                
        elif len(newEdgeCons) > 0:
            while not IsCardinal:
                a, b, (i1, j1), (i2, j2), t = newEdgeCons[0]
                
                FlagChild1 = False
                FlagChild2 = False
                
                Child1_not_expanded = False
                Child2_not_expanded = False
            
            
                # Разбиваем CT на ноды A и B, разрешая реберный конфликт 
            
                tmp = s.edgeCons.copy()
                if a in tmp:
                    tmp[a].append(((i1, j1), (i2, j2), t))   
                else:
                    tmp[a] = [((i1, j1), (i2, j2), t)]
            
                A = HighNode(
                    vertexCons=s.vertexCons,
                    edgeCons=tmp,
                    sol=s.sol,
                    parent=s,
                    k=k,
                )
            
                vc = []
                if a in A.vertexCons:
                    vc = A.vertexCons[a]
            
                planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vc, A.edgeCons[a])
                result = planner.FindPath()
                if result[0]:
                    path = MakePath(result[1])
                    A.sol[a], _ = path
                    A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                    
                    if  A.g > s.g:
                        FlagChild1 = True #OPEN.AddNode(A)
                    if not CLOSED.WasExpanded(A):
                        Child1_not_expanded = True
                        
                    k += 1
                
                tmp = s.edgeCons.copy()
                if b in tmp:
                    tmp[b].append(((i1, j1), (i2, j2), t))   
                else:
                    tmp[b] = [((i1, j1), (i2, j2), t)]
            
                B = HighNode(
                    vertexCons=s.vertexCons,
                    edgeCons=tmp,
                    sol=s.sol,
                    parent=s,
                    k=k,
                )
            
                vc = []
                if b in B.vertexCons:
                    vc = B.vertexCons[b]
            
                planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], vc, B.edgeCons[b])
                result = planner.FindPath()
                if result[0]:
                    path = MakePath(result[1])
                    B.sol[b], _ = path
                    B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                    if  B.g > s.g:
                        FlagChild2 = True #OPEN.AddNode(B)
                    if not CLOSED.WasExpanded(B):
                        Child2_not_expanded = True
                    k += 1
                if FlagChild1 and FlagChild2:
                    IsCardinal = True
                    if  Child1_not_expanded and  Child2_not_expanded:
                        OPEN.AddNode(A)
                        OPEN.AddNode(B)
                elif index == len(newVertexCons)-1:
                    if  Child1_not_expanded and  Child2_not_expanded:
                        OPEN.AddNode(A)
                        OPEN.AddNode(B)
                else: 
                    index+=1
                
    return (False, None, OPEN, CLOSED)